In [84]:
import pandas as pd
import numpy as np 
import pathlib
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import TimeDistributed
from keras.optimizers import Adam

In [87]:
# Loading the dataset 
data_dir = pathlib.Path('/home/lv11/Documents/ProyectosPython/sentimentAnalysis/train')
nf = pd.read_csv(data_dir / 'tweetsDataset1.csv',skiprows=1,names=['Message','Target'])
#print(nf.head(20))


In [89]:
nlp = English()
stop_words = list(STOP_WORDS)

messages = nf['Message']
labels = nf['Target']
#print(stop_words)

def spacy_tokenizer(sentence):
    tokens = nlp(sentence)
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuation ]
    return tokens

In [91]:
def vectorization(tweets):
    vector = CountVectorizer(tokenizer=spacy_tokenizer)
    frequency_matrix = vector.fit_transform(tweets)
    sum_frequencies = np.sum(frequency_matrix,axis=0)
    frequency = np.squeeze(np.asarray(sum_frequencies))
    frequency_dataFrame = pd.DataFrame([frequency],columns=vector.get_feature_names()).transpose()
    return frequency_dataFrame

In [93]:
def splitting(messages,labels):
    X_train,X_test,y_train,y_test = train_test_split(messages,labels,test_size=0.25,random_state=42,shuffle=True)
    return X_train,X_test,y_train,y_test

In [95]:
def tokenization_tweets(dataset,features):
    tokenization = TfidfVectorizer(max_features=features)
    tokenization.fit(dataset)
    dataset_transformed = tokenization.transform(dataset).toarray()
    return dataset_transformed

In [97]:
def train(X_train_mod, y_train, features, shuffle, drop, layer1, layer2, epoch, lr, epsilon, validation):
    model_nn = Sequential()
    model_nn.add(Dense(layer1, input_shape=(features,), activation='relu'))
    model_nn.add(Dropout(drop))
    model_nn.add(Dense(layer2, activation='sigmoid'))
    model_nn.add(Dropout(drop))
    model_nn.add(Dense(3, activation='softmax'))
    
    optimizer = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon,decay=0.0, amsgrad=False)
    model_nn.compile(loss='sparse_categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])
    model_nn.fit(np.array(X_train_mod), y_train,
                 batch_size=32,
                 epochs=epoch,
                 verbose=1,
                 validation_split=validation,
                 shuffle=True)
    return model_nn

In [99]:
def test(X_test,model):
    predict = model.predict(X_test)
    return predict

In [101]:
def modelo1(X_train,y_train):
    features = 1554 #1727 # puede ser 3727 o un valor asi que vi como len() del array de X_train 
    shuffle = True
    drop = 0.5
    layer1 = 512
    layer2 = 256
    epoch = 10
    lr = 0.002
    epsilon = 1e-9
    validation = 0.1
    X_train_mod = tokenization_tweets(X_train,features)
    modelo = train(X_train_mod, y_train,features,shuffle,drop,layer1,layer2,epoch,lr,epsilon,validation)
    return modelo 

In [103]:
frequency_dataFrame = vectorization(messages)

In [105]:
X_train,X_test,y_train,y_test = splitting(messages,labels)

In [107]:
modelo = modelo1(X_train,y_train)

Train on 1554 samples, validate on 173 samples
Epoch 1/10
1554/1554 [==============================] - 1s 915us/step - loss: 0.6877 - accuracy: 0.6197 - val_loss: 0.3283 - val_accuracy: 0.8728
Epoch 2/10
1554/1554 [==============================] - 1s 692us/step - loss: 0.2361 - accuracy: 0.9048 - val_loss: 0.2067 - val_accuracy: 0.8960
Epoch 3/10
1554/1554 [==============================] - 1s 801us/step - loss: 0.1199 - accuracy: 0.9582 - val_loss: 0.2791 - val_accuracy: 0.8728
Epoch 4/10
1554/1554 [==============================] - 1s 784us/step - loss: 0.0721 - accuracy: 0.9730 - val_loss: 0.3040 - val_accuracy: 0.8786
Epoch 5/10
1554/1554 [==============================] - 1s 765us/step - loss: 0.0586 - accuracy: 0.9794 - val_loss: 0.3154 - val_accuracy: 0.8555
Epoch 6/10
1554/1554 [==============================] - 1s 731us/step - loss: 0.0379 - accuracy: 0.9858 - val_loss: 0.3839 - val_accuracy: 0.8555
Epoch 7/10
1554/1554 [==============================] - 1s 784us/step - loss:

In [ ]:
modelo.summary()

In [115]:
pred = modelo.predict(X_test)

ValueError: Error when checking input: expected dense_46_input to have shape (1554,) but got array with shape (1,)